----

- xpath、css 选择器

  对于网页的节点来说，它可以定义 id、class 或其他属性。而且节点之间还有层次关系，***`在网页中可以通过 XPath 或 CSS 选择器来定位一个或多个节点`***。那么，在页面解析时，利用 XPath 或 CSS 选择器来提取某个节点，然后再调用相应方法获取它的正文内容或者属性，不就可以提取我们想要的任意信息了吗？

- Beautiful、lxml、pyquery、parsel 解析库

- re 提前页面信息

----

lxml 配合 xpath

pyquery 配合 css selector

parsel 配合 xpath & css selector

----

BeautifulSoup - 两种获取信息的方法
  1. `find()` & `find_all()`【前者返回第一个匹配结果，后者返回所有匹配到的结果列表】
  2. CSS选择器 `soup.select(<css selector>)`【返回所有匹配到的结果列表】

----

# 各种采集数据的方式

## 一、BeautifulSoup 


<img alt="图 1" src="../images/92a6956d27084bccba5ca2a65a02aa8f7955d6ed1b9da560b1b3acd46a1e1db3.png" width=75%>  

> [[Python3 网络爬虫开发实战] 4.2 - 使用 Beautiful Soup](https://cuiqingcai.com/5548.html)

简单来说，***`Beautiful Soup 就是 Python 的一个 HTML 或 XML 的解析库`***，可以用它来方便地从网页中提取数据。


In [54]:
import json
import requests
from bs4 import BeautifulSoup

url = 'https://pvp.qq.com/web201605/herolist.shtml'

res = requests.get(url)
res.encoding = 'gbk'    # 改变编码格式
html = res.text
with open('./王者荣耀/data.html', 'w', encoding = 'utf-8') as f:
    f.write(html)
soup = BeautifulSoup(html, 'lxml')
hero_url_list = []

'''
### 通过 CSS 选择器
for item in soup.select('.herolist.clearfix a'):
    name = item.get_text()
    url = 'https://pvp.qq.com/web201605/' + item.attrs['href']
    # soup.select('.herolist.clearfix a')[0]['href']
    hero_url_list.append({'name':name, 'url':url})
hero_url_list[:5]
'''

### 通过 find_all 寻找节点
### 两种方法均可正常使用，根据习惯选择某一种方法即可
for a in soup.find(class_="herolist").find_all('a'):
    href = 'https://pvp.qq.com/web201605/' + a.attrs['href']
    name = a.get_text()
    hero_url_list.append({'name':name, 'url':url})
# 数据预览
hero_url_list[:5]

[{'name': '云中君', 'url': 'https://pvp.qq.com/web201605/herolist.shtml'},
 {'name': '瑶', 'url': 'https://pvp.qq.com/web201605/herolist.shtml'},
 {'name': '盘古', 'url': 'https://pvp.qq.com/web201605/herolist.shtml'},
 {'name': '猪八戒', 'url': 'https://pvp.qq.com/web201605/herolist.shtml'},
 {'name': '嫦娥', 'url': 'https://pvp.qq.com/web201605/herolist.shtml'}]

**JSON 格式存储**

In [53]:
with open('./王者荣耀/data.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(hero_url_list, indent = 4, ensure_ascii=False))

# 关联知识
# 文件读取写入的 r w a ...的意思

## 二、lxml & XPath

> 乱码问题
> 
> [requests+lxml+xpath爬取豆瓣电影](https://cloud.tencent.com/developer/article/1686201)

```Python
print(len(html.xpath('.//*')))
print(len(html.xpath('//*')))

>>> 469
>>> 470
```

`.//` 当前节点下的所有节点
`//` 自动补齐的所有节点（如果是从 ul 节点获取所有节点 则包括 补齐的 html body head 等节点

In [17]:
import requests
from lxml import etree
url = 'https://pvp.qq.com/web201605/herolist.shtml'
res = requests.get(url)
res.encoding = 'gbk'    # 改变编码格式
html = etree.HTML(res.text)
# result = etree.tostring(html, encoding='utf-8') # 注意 encoding
# print(result.decode('utf-8'))

In [25]:
# 不知道有没有什么优化方法
result = html.xpath('//ul[contains(@class, "herolist") and contains(@class, "clearfix")]')[0].xpath('.//li/a')
print(len(result))

hero_url_list = []
for a in result:
    url = 'https://pvp.qq.com/web201605/' + a.xpath('./@href')[0]
    name = a.xpath('./text()')[0]
    hero_url_list.append({'name':name, 'url':url})
hero_url_list[:5]       # 数据预览

93


[{'name': '云中君', 'url': 'https://pvp.qq.com/web201605/herodetail/506.shtml'},
 {'name': '瑶', 'url': 'https://pvp.qq.com/web201605/herodetail/505.shtml'},
 {'name': '盘古', 'url': 'https://pvp.qq.com/web201605/herodetail/529.shtml'},
 {'name': '猪八戒', 'url': 'https://pvp.qq.com/web201605/herodetail/511.shtml'},
 {'name': '嫦娥', 'url': 'https://pvp.qq.com/web201605/herodetail/515.shtml'}]

## 三、PyQuery & CSS selector

## 四、Parsel & XPath + CSS Selector

## 五、re 正则表达式

## 六、Selenium & 

# 项目

通过以上几种方式获得英雄信息有所缺失，王者荣耀官网中现有 120 名英雄，但是以上几种方式获得的英雄数量均为 93 ，所有可以判断网页可能存在 **动态内容**

> 注
> 一至五的五种方法均是通过 `requests` 库获取网页源代码，返回的 HTML内容 和 网页展现给我们的内容是不同的
> 第六种方法暂未测速判断是否可以获取正常数量的英雄信息

In [34]:
import requests
res = requests.get('https://pvp.qq.com/web201605/js/herolist.json')
herolist_json = res.json()

hero_url_list = []
for item in herolist_json:
    herourl  = 'https://pvp.qq.com/web201605/herodetail/{}.shtml'.format(item['ename'])
    heroname = item['cname']
    hero_url_list.append({'name':heroname, 'url':herourl})
len(hero_url_list), hero_url_list[:5]   # 数据预览

(110,
 [{'name': '廉颇', 'url': 'https://pvp.qq.com/web201605/herodetail/105.shtml'},
  {'name': '小乔', 'url': 'https://pvp.qq.com/web201605/herodetail/106.shtml'},
  {'name': '赵云', 'url': 'https://pvp.qq.com/web201605/herodetail/107.shtml'},
  {'name': '墨子', 'url': 'https://pvp.qq.com/web201605/herodetail/108.shtml'},
  {'name': '妲己', 'url': 'https://pvp.qq.com/web201605/herodetail/109.shtml'}])

In [30]:
herolist_json[:2]

[{'ename': 105,
  'cname': '廉颇',
  'title': '正义爆轰',
  'new_type': 0,
  'hero_type': 3,
  'skin_name': '正义爆轰|地狱岩魂',
  'moss_id': 3627},
 {'ename': 106,
  'cname': '小乔',
  'title': '恋之微风',
  'new_type': 0,
  'hero_type': 2,
  'skin_name': '恋之微风|万圣前夜|天鹅之梦|纯白花嫁|缤纷独角兽',
  'moss_id': 3644}]